In [1]:
library(Seurat)
library(glmGamPoi)
library(limma)
library(future)
library(qs)

Attaching SeuratObject

qs v0.25.1.



In [2]:
plan("multicore", workers = 16)
plan()

multicore:
- args: function (..., envir = parent.frame(), workers = 16)
- tweaked: TRUE
- call: plan("multicore", workers = 16)

In [3]:
library(parallel)
ncores <- detectCores()-1
cl <- makeCluster(ncores)
print(ncores)

[1] 15


In [4]:
# Load four Seurat objects and merge 
A <- qread("COH096.seurat_v2.qs")
B <- qread("COH098.seurat_v2.qs")
C <- qread("COH099.seurat_v2.qs")
D <- qread("COH100.seurat_v2.qs")

#A.encode <- readRDS("COH096.Singler.encode.RDS")
#B.encode <- readRDS("COH098.Singler.encode.RDS")
#C.encode <- readRDS("COH099.Singler.encode.RDS")
#D.encode <- readRDS("COH100.Singler.encode.RDS")
#A$encode <- A.encode$pruned.labels
#B$encode <- B.encode$pruned.labels
#C$encode <- C.encode$pruned.labels
#D$encode <- D.encode$pruned.labels

big.seurat <- merge(A, y=c(B, C, D))

rm(A, B, C, D)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,6259588,334.3,9674024,516.7,8140008,434.8
Vcells,2131999239,16265.9,5877426834,44841.3,5666752273,43233.9


In [21]:
dim(big.seurat)

[1]  21244 219532

In [6]:
qsave(big.seurat, file="1_big.seurat_v2.qs")

In [20]:
big.seurat <- qread(file="1_big.seurat_v2.qs", nthreads=16)

In [22]:
table(big.seurat$Sample)


 16-0012853  16-0015453  16-0035651  16-0060770  16-0062458  17-0023835 
        705        3121         181        1730        2715        2417 
  17387-05A   17387-07A   17387-12A   17387-14A   17387-18A   17387-20A 
        877         234        1772        2306        4062        1042 
  17387-21A   17387-23A   17387-34A   17387-35A   17387-40A   17387-41A 
       3467         759        1718        1653        3568        2967 
  17387-42A   17387-51A   17387-52A   17387-55A   17387-57A   17387-60A 
        217        2459         477        2353        1636        1705 
  17387-64A   17387-68A   17387-73A   17387-80A   17387-81A   17387-84A 
       2435        2121        1139        3200        2371        1552 
  17387-90A  18-0003513  18-0007107  18-0007119  18-0010033  18-0013087 
       2612        1916        1354        2791        1426        1905 
 18-0015075  18-0015502  18-0019386  18-0021297  18-0022132  18-0023702 
       1128        3979        1957         667   

In [23]:
#big.seurat <- qread(file="1_big.seurat_v2.rds"); print("Done loading")
bs.list <- SplitObject(big.seurat, split.by="Sample")
rm(big.seurat)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,6305187,336.8,9674024,516.7,9674024,516.7
Vcells,2137582910,16308.5,8125187815,61990.3,7511208178,57306.0


In [24]:
bs.list <- mclapply(bs.list, FUN = NormalizeData, normalization.method = "LogNormalize",
                     scale.factor = 10000, mc.cores = ncores)

In [25]:
bs.list <- mclapply(bs.list, FUN = FindVariableFeatures, selection.method = "vst", nfeatures = 2000, mc.cores = ncores)

In [5]:
bs.list <- mclapply(bs.list, FUN = ScaleData, mc.cores = 1)

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scaling data matrix

Centering and scalin

In [6]:
qsave(bs.list, file="2_big.seurat.splitlist.norm_v2.qs", nthreads=16)

In [4]:
bs.list <- qread(file="2_big.seurat.splitlist.norm_v2.qs", nthreads=16)

In [7]:
bs.list <- mclapply(bs.list, FUN=RunPCA, verbose=FALSE, mc.cores=1)

Warning message in irlba(A = t(x = object), nv = npcs, ...):
“You're computing too large a percentage of total singular values, use a standard svd instead.”


In [8]:
pdf("Figures/Big.seurat_split_lapply_ElbowPlot.pdf", height=4, width=4)
lapply(X=bs.list, FUN=ElbowPlot, 50)
dev.off()

pdf("Figures/Big.seurat_split_lapply_PCAPlot.pdf", height=4, width=4)
lapply(X=bs.list, FUN=DimPlot)
dev.off()

pdf("Figures/Big.seurat_split_lapply_VlnPlot.pdf", height=4, width=10)
lapply(X=bs.list, FUN=VlnPlot, features = c("nFeature_RNA", "nCount_RNA", "Percent.Mitochondria"), 
       ncol = 3, pt.size=0.25, assay='SCT')
dev.off()

$`AB-24A`

$`AB-33D`

$`19-0016739`

$`AB-74A`

$`19-0016212`

$`AB-67A`

$`AB-82A`

$`AB-16D`

$`18-0015502`

$`AB-52A`

$AB14C

$`18-0023702`

$`17-0023835`

$`AB-72A`

$`AB-80A`

$`19-0032047`

$`18-0003513`

$`18-0021297`

$`20-0010478`

$`19-0023125`

$`20-0003838`

$`20-0017984`

$`19-0004870`

$`20-0016800`

$`20-0004455`

$`AB-11O`

$`16-0012853`

$`18-0010033`

$`20-0002786`

$`18-0007119`

$`AB-38A`

$`AB-11K`

$`18-0026838`

$`18-0026653`

$`19-0023420`

$`19-0000610`

$`18-0013087`

$`18-0015075`

$`16-0062458`

$`20-0021810`

$`19-0019883`

$`19-0026358`

$`AB--62B`

$`19-0011792`

$`19-0027486`

$`20-0021233`

$`16-0060770`

$`18-0007107`

$`19-8028436`

$`16-0015453`

$`17387-12A`

$`17387-52A`

$`18-0019386`

$`17387-73A`

$`17387-81A`

$`17387-55A`

$`17387-34A`

$`17387-40A`

$`17387-60A`

$`17387-80A`

$`17387-05A`

$`17387-57A`

$`17387-23A`

$`17387-21A`

$`17387-41A`

$`17387-84A`

$`17387-64A`

$`17387-51A`

$`17387-68A`

$`19-0023592A`

$`17387-20A`

$`17387-14A

png 
  2

$`AB-24A`

$`AB-33D`

$`19-0016739`

$`AB-74A`

$`19-0016212`

$`AB-67A`

$`AB-82A`

$`AB-16D`

$`18-0015502`

$`AB-52A`

$AB14C

$`18-0023702`

$`17-0023835`

$`AB-72A`

$`AB-80A`

$`19-0032047`

$`18-0003513`

$`18-0021297`

$`20-0010478`

$`19-0023125`

$`20-0003838`

$`20-0017984`

$`19-0004870`

$`20-0016800`

$`20-0004455`

$`AB-11O`

$`16-0012853`

$`18-0010033`

$`20-0002786`

$`18-0007119`

$`AB-38A`

$`AB-11K`

$`18-0026838`

$`18-0026653`

$`19-0023420`

$`19-0000610`

$`18-0013087`

$`18-0015075`

$`16-0062458`

$`20-0021810`

$`19-0019883`

$`19-0026358`

$`AB--62B`

$`19-0011792`

$`19-0027486`

$`20-0021233`

$`16-0060770`

$`18-0007107`

$`19-8028436`

$`16-0015453`

$`17387-12A`

$`17387-52A`

$`18-0019386`

$`17387-73A`

$`17387-81A`

$`17387-55A`

$`17387-34A`

$`17387-40A`

$`17387-60A`

$`17387-80A`

$`17387-05A`

$`17387-57A`

$`17387-23A`

$`17387-21A`

$`17387-41A`

$`17387-84A`

$`17387-64A`

$`17387-51A`

$`17387-68A`

$`19-0023592A`

$`17387-20A`

$`17387-14A

png 
  2

$`AB-24A`

$`AB-33D`

$`19-0016739`

$`AB-74A`

$`19-0016212`

$`AB-67A`

$`AB-82A`

$`AB-16D`

$`18-0015502`

$`AB-52A`

$AB14C

$`18-0023702`

$`17-0023835`

$`AB-72A`

$`AB-80A`

$`19-0032047`

$`18-0003513`

$`18-0021297`

$`20-0010478`

$`19-0023125`

$`20-0003838`

$`20-0017984`

$`19-0004870`

$`20-0016800`

$`20-0004455`

$`AB-11O`

$`16-0012853`

$`18-0010033`

$`20-0002786`

$`18-0007119`

$`AB-38A`

$`AB-11K`

$`18-0026838`

$`18-0026653`

$`19-0023420`

$`19-0000610`

$`18-0013087`

$`18-0015075`

$`16-0062458`

$`20-0021810`

$`19-0019883`

$`19-0026358`

$`AB--62B`

$`19-0011792`

$`19-0027486`

$`20-0021233`

$`16-0060770`

$`18-0007107`

$`19-8028436`

$`16-0015453`

$`17387-12A`

$`17387-52A`

$`18-0019386`

$`17387-73A`

$`17387-81A`

$`17387-55A`

$`17387-34A`

$`17387-40A`

$`17387-60A`

$`17387-80A`

$`17387-05A`

$`17387-57A`

$`17387-23A`

$`17387-21A`

$`17387-41A`

$`17387-84A`

$`17387-64A`

$`17387-51A`

$`17387-68A`

$`19-0023592A`

$`17387-20A`

$`17387-14A

png 
  2

In [8]:
bs.list <- mclapply(bs.list, FUN=FindNeighbors, reduction = "pca", dims = 1:50, nn.eps = 0.5, verbose=FALSE, mc.cores=1)

In [10]:
bs.list <- mclapply(bs.list, FUN=FindClusters, resolution = 0.5, n.start = 10, verbose=FALSE, mc.cores=1)

Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”
Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "igno

In [11]:
bs.list <- mclapply(bs.list, FUN=RunUMAP, dims=1:50, verbose=FALSE, mc.cores=1)

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”


In [12]:
qsave(bs.list, file="2_big.seurat.splitlist.norm_v2.qs", nthreads=16)

In [4]:
bs.list <- qread(file="2_big.seurat.splitlist.norm_v2.qs", nthreads=16)

In [10]:
library(data.table)

In [11]:
#### Export counts matrix for SingleR ####
dir.create("./counts")
for (i in 1:length(bs.list)) {
    counts <- data.frame(GetAssayData(object = bs.list[[i]], slot = "counts"))
    fname <- paste0("./counts/", names(bs.list)[i], ".counts.txt")
    fwrite(counts, file=fname, sep = '\t', row.names = T, col.names = T, nThread = 16)
}

Warning message in dir.create("./counts"):
“'./counts' already exists”


In [18]:
write.table(file="./counts/Samples.v2.txt", sort(names(bs.list)), sep="\n", quote=F, col.names=F, row.names=F)

In [28]:
#### Import SingleR annotations and integrate ####
for (i in 1:length(bs.list)) {
  encode <- readRDS(file=paste0("./SingleR/", names(bs.list)[i], ".encode.RDS"))
  hpca <- readRDS(file=paste0("./SingleR/", names(bs.list)[i], ".hpca.RDS"))

  x <- match(make.names(colnames(bs.list[[i]])), rownames(encode))
  bs.list[[i]]$'encode' <- encode$pruned.labels[x]
  bs.list[[i]]$'encode.scores' <- encode$scores[x]

  x <- match(make.names(colnames(bs.list[[i]])), rownames(hpca))
  bs.list[[i]]$'hpca' <- hpca$pruned.labels[x]
  bs.list[[i]]$'hpca.scores' <- hpca$scores[x]
} 

In [31]:
qsave(bs.list, file="2.1_big.seurat.splitlist.norm_SingleR_v2.qs", nthreads=16)